# Notes

Emm I read the pdf and it's a bit hard to split this project into tasks tbh. From what I understood the goal, in short, is to achieve the best possible accuracy with least amount of features used. Whereas there is some reward for higher accuracy and penalty for number of features used so we need to find a proper balance. 5 different approaches should be used. It is a binary classification task. So I think we could:

1. Find some best candidates methods for binary classification tasks with numeric features.
2. One approach would be to iterate over the number of features starting from 1 (xd) and find the number that achieves the best score for each of the chosen methods
3. We could also do some feature selection methods and try to find the best number of features that way.
4. We could do some ensembling, maybe combining  models from the above

# TODO:

1. Find some good candidates for binary classification including ensembles
    1. 5 different models
2. Write a pipeline for testing and analysis of the results
3. Write the report
4. Prepare the presentation

In [47]:
import pickle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import numpy as np

# Load Data

load the training and test data in a format specified by task description

In [5]:
with open('./Dataset/x_test.txt') as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open('./Dataset/x_train.txt') as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open('./Dataset/y_train.txt') as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]



In the X_test there are 500 features and 5000 observations, y_train contains 5000 values, X_train contains 500 features and 5000 observations

In [6]:
print('X_test datapoints:',len(X_test)," features:",len(X_test[0]))
print('X_train datapoints:',len(X_train)," features:",len(X_train[0]))
print('y_train datapoints:',len(y_train))


X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000


# Experiment

## Models

In [23]:
# TODO: Put here at least 5 different models, which can be including ensembles
models = [LinearDiscriminantAnalysis(),QuadraticDiscriminantAnalysis()]
# TODO: Put here corresponding model names, names You would like to see on charts etc
modelNames = ['xd']
# TODO: Put here selection techniques (Can be the same as in data exploration project 4 (I will do it - Patryk))
featureSelectors=[]
featureSelectorsNames=['xd']

# Parameters

In [8]:
# TODO: Put here model parameters in the following format
modelParameters=[{str(modelNames[0]):{'param1':5}}]
# TODO: A dictionary with experiment function parameters (maybe things such as number of repetitions, maybe some parameters for )
experimentParameters = [{"param1":5},]
# TODO: Feature selector parameters
featureSelectorsParameters=[{str(featureSelectorsNames[0]):{'param1':5}}]

## Main experiment loop

In [73]:
def performExperiment(param1,X_train,y_train,X_test,model):
    model.fit(X_train,y_train)
    result = model.predict_proba(X_test)
    # Take only 1000 of the highest class 1 probabilities
    # TODO: Turn it into a dataframe and keep track of the indexes
    finalResult = np.sort(result[:,1])[::-1][:1000]
    
    return finalResult

## Conducting the experiment

collect info about features used and the amount of predicted positive class

In [74]:
%%time
results=[]
for parameters in experimentParameters:
    for model in models:
        result = performExperiment(**parameters,X_train=X_train,y_train=y_train,X_test=X_test,model=model)
        results.append(result)
with open("./Results/results", "wb") as f:
    pickle.dump(results, f)

c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


CPU times: total: 2.73 s
Wall time: 4.21 s


#### The performance of the model could be measure in terms of: 

1. Taking the average from the proba array (array returned by experiment function)
2. The higher the average the better the model

# Results

In [ ]:
with open(r"./Results/results", "rb") as input_file:
     results = pickle.load(input_file)

# TODO: Process the results, some charts etc, maybe some correlations, line plots etc